In [1]:
import sqlite3
import json
import os
from datetime import datetime, timezone, timedelta

conn = sqlite3.connect('../DB/analytics.db')
cur = conn.cursor()

In [2]:
def load_json(filename):
    filepath = os.path.join("..", "sample_analytics_dataset", filename)
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)


In [3]:
accounts = load_json('sample_analytics.accounts.json')

In [4]:
for account in accounts:
    cur.execute(
        "INSERT OR IGNORE INTO account (account_id, limit_amount) VALUES (?, ?)",
        (account['account_id'], account['limit'])
    )
conn.commit()

In [5]:
for account in accounts:
    for product in account['products']:
        cur.execute(
            "INSERT OR IGNORE INTO product (product_name) VALUES (?)",
            (product,)
        )
        cur.execute(
            "INSERT OR IGNORE INTO account_product (account_id, product_name) VALUES (?, ?)",
            (account['account_id'], product)
        )
conn.commit()

In [6]:
customers = load_json('sample_analytics.customers.json')

In [7]:
def parse_epoch(birthdate_field):
    try:
        date_data = birthdate_field['$date']

        if isinstance(date_data, str):
            return datetime.strptime(date_data[:10], '%Y-%m-%d').date()

        # Epoch timestamp, vi la forma de hacerlo aqui https://stackoverflow.com/questions/17231711/how-to-create-datetime-from-a-negative-epoch-in-python
        elif isinstance(date_data, dict) and '$numberLong' in date_data:
            millis = int(date_data['$numberLong'])
            seconds = millis / 1000
            base = datetime(1970, 1, 1, tzinfo=timezone.utc)
            return (base + timedelta(seconds=seconds)).date()

    except Exception as e:
        print(f"Error parsing date: {birthdate_field} — {e}")
        return None

In [8]:
for customer in customers:
    birthdate = parse_epoch(customer['birthdate'])
    birthdate_str = birthdate.isoformat() if birthdate else None
    
    cur.execute(
        "INSERT OR IGNORE INTO customer (username, name, address, birthdate, email) VALUES (?, ?, ?, ?, ?)",
        (customer['username'], customer['name'], customer['address'], birthdate_str, customer['email'])
    )
conn.commit()

In [9]:
for customer in customers:
    for account_id in customer['accounts']:
        cur.execute(
            "INSERT OR IGNORE INTO customer_account (username, account_id) VALUES (?, ?)",
            (customer['username'], account_id)
        )
conn.commit()

In [10]:
for customer in customers:
    tier_details = customer.get('tier_and_details', {})
    for tier_id, tier_info in tier_details.items():
        cur.execute(
            "INSERT OR IGNORE INTO tier (tier_id, username, tier_name, active) VALUES (?, ?, ?, ?)",
            (tier_id, customer['username'], tier_info['tier'], tier_info['active'])
        )
        
        for benefit in tier_info.get('benefits', []):
            cur.execute(
                "INSERT OR IGNORE INTO benefit (benefit_name) VALUES (?)",
                (benefit,)
            )
            
            cur.execute(
                "INSERT OR IGNORE INTO tier_benefit (tier_id, benefit_name) VALUES (?, ?)",
                (tier_id, benefit)
            )
conn.commit()

In [11]:
transactions = load_json('sample_analytics.transactions.json')

In [12]:
for entry in transactions:
    account_id = entry["account_id"]
    
    for tx in entry["transactions"]:
        tx_date = parse_epoch(tx["date"])
        tx_date_str = tx_date.isoformat() if birthdate else None
        amount = tx["amount"]
        code = tx["transaction_code"]
        symbol = tx.get("symbol", None)
        price = float(tx["price"])
        total = float(tx["total"])

        cur.execute("""
            INSERT INTO transactions (
                account_id, transaction_date, amount,
                transaction_code, symbol, price, total
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            account_id, tx_date_str, amount,
            code, symbol, price, total
        ))

conn.commit()

In [13]:
conn.close() 